In [3]:
import pandas as pd

In [1]:
def map_to_macrocategory(category):
    if category in ['BEVERAGES', 'BREAD/BAKERY', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 
                    'GROCERY I', 'GROCERY II', 'MEATS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE', 'SEAFOOD']:
        return 'Alimentos e Bebidas'
    elif category in ['BABY CARE', 'BEAUTY', 'PERSONAL CARE']:
        return 'Cuidados Pessoais e Saúde'
    elif category in ['CLEANING', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES', 
                      'HOME CARE', 'LAWN AND GARDEN']:
        return 'Casa e Jardim'
    elif category in ['LADIESWEAR', 'LINGERIE']:
        return 'Moda e Vestuário'
    elif category == 'AUTOMOTIVE':
        return 'Automotivo'
    elif category == 'PET SUPPLIES':
        return 'Animais de Estimação'
    elif category in ['BOOKS', 'MAGAZINES', 'CELEBRATION', 'PLAYERS AND ELECTRONICS', 'HARDWARE']:
        return 'Lazer, Tecnologia e Ferramentas'
    elif category == 'LIQUOR,WINE,BEER':
        return 'Bebidas Alcoólicas'
    elif category == 'SCHOOL AND OFFICE SUPPLIES':
        return 'Escola e Escritório'
    else:
        return 'Outros'

In [19]:
treino = pd.read_csv('Data/train.csv.gz', compression='gzip')
treino= treino.drop(columns=['id'])
transactions = pd.read_csv('Data/transactions.csv')
stores = pd.read_csv('Data/stores.csv')
oil = pd.read_csv('Data/oil.csv')
holidays = pd.read_csv('Data/holidays_events.csv')

In [30]:
treino.describe

<bound method NDFrame.describe of                date  store_nbr                      family     sales  \
0        2013-01-01          1                  AUTOMOTIVE     0.000   
1        2013-01-01          1                   BABY CARE     0.000   
2        2013-01-01          1                      BEAUTY     0.000   
3        2013-01-01          1                   BEVERAGES     0.000   
4        2013-01-01          1                       BOOKS     0.000   
...             ...        ...                         ...       ...   
3000883  2017-08-15          9                     POULTRY   438.133   
3000884  2017-08-15          9              PREPARED FOODS   154.553   
3000885  2017-08-15          9                     PRODUCE  2419.729   
3000886  2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
3000887  2017-08-15          9                     SEAFOOD    16.000   

         onpromotion  
0                  0  
1                  0  
2                  0  
3        

In [35]:
dados = pd.merge(treino, transactions, on=['date', 'store_nbr'], how='left')
dados= pd.merge(dados, oil, on=['date'], how= 'left')

In [36]:
dados.describe

<bound method NDFrame.describe of                date  store_nbr                      family     sales  \
0        2013-01-01          1                  AUTOMOTIVE     0.000   
1        2013-01-01          1                   BABY CARE     0.000   
2        2013-01-01          1                      BEAUTY     0.000   
3        2013-01-01          1                   BEVERAGES     0.000   
4        2013-01-01          1                       BOOKS     0.000   
...             ...        ...                         ...       ...   
3000883  2017-08-15          9                     POULTRY   438.133   
3000884  2017-08-15          9              PREPARED FOODS   154.553   
3000885  2017-08-15          9                     PRODUCE  2419.729   
3000886  2017-08-15          9  SCHOOL AND OFFICE SUPPLIES   121.000   
3000887  2017-08-15          9                     SEAFOOD    16.000   

         onpromotion  transactions  dcoilwtico  
0                  0           NaN         NaN  
1  

In [38]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 7 columns):
 #   Column        Dtype  
---  ------        -----  
 0   date          object 
 1   store_nbr     int64  
 2   family        object 
 3   sales         float64
 4   onpromotion   int64  
 5   transactions  float64
 6   dcoilwtico    float64
dtypes: float64(3), int64(2), object(2)
memory usage: 160.3+ MB


In [7]:
dados["family"].unique()

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

In [41]:
null_counts = transactions.isnull().sum()
print(null_counts)

date            0
store_nbr       0
transactions    0
dtype: int64
